# Momentum Trading Strategy

## Overview
This notebook implements a **cross-sectional momentum strategy** from data acquisition through backtesting and performance analysis.

### Strategy Logic
- **Universe**: S&P 500 constituents
- **Signal**: 12-month returns, excluding the most recent month (12-1 momentum)
- **Rebalancing**: Monthly
- **Portfolio**: Long top decile, short bottom decile

### Contents
1. [Data Acquisition](#1.-Data-Acquisition)
2. [Feature Engineering](#2.-Feature-Engineering)
3. [Signal Construction](#3.-Signal-Construction)
4. [Backtesting](#4.-Backtesting)
5. [Performance Analysis](#5.-Performance-Analysis)
6. [Risk Analysis](#6.-Risk-Analysis)

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)
plt.style.use('seaborn-v0_8-whitegrid')

# Random seed for reproducibility
np.random.seed(42)

## 1. Data Acquisition

For this demonstration, we generate synthetic price data that mimics real market characteristics:
- Log-normal returns
- Volatility clustering
- Cross-sectional correlation structure

In [ ]:
def generate_synthetic_prices(n_assets=100, n_days=2520, annual_vol=0.25):
    """
    Generate synthetic price data for backtesting.
    
    Parameters:
    -----------
    n_assets : int
        Number of assets in the universe
    n_days : int
        Number of trading days (2520 ≈ 10 years)
    annual_vol : float
        Average annual volatility
        
    Returns:
    --------
    pd.DataFrame
        Price data with DatetimeIndex and asset columns
    """
    # Generate date index
    dates = pd.date_range(start='2014-01-01', periods=n_days, freq='B')
    
    # Daily volatility from annual
    daily_vol = annual_vol / np.sqrt(252)
    
    # Generate correlated returns with market factor
    market_returns = np.random.normal(0.0003, daily_vol, n_days)
    
    # Asset-specific parameters
    betas = np.random.uniform(0.5, 1.5, n_assets)
    alphas = np.random.normal(0, 0.0001, n_assets)
    idio_vols = np.random.uniform(0.01, 0.03, n_assets)
    
    # Generate returns
    returns = np.zeros((n_days, n_assets))
    for i in range(n_assets):
        idio_returns = np.random.normal(0, idio_vols[i], n_days)
        returns[:, i] = alphas[i] + betas[i] * market_returns + idio_returns
    
    # Convert to prices
    prices = 100 * np.exp(np.cumsum(returns, axis=0))
    
    # Create DataFrame
    columns = [f'ASSET_{i:03d}' for i in range(n_assets)]
    return pd.DataFrame(prices, index=dates, columns=columns)

# Generate data
prices = generate_synthetic_prices()
print(f"Price data shape: {prices.shape}")
print(f"Date range: {prices.index[0]} to {prices.index[-1]}")
prices.head()

In [ ]:
# Visualize sample prices
fig, ax = plt.subplots(figsize=(12, 6))
sample_assets = np.random.choice(prices.columns, 5, replace=False)
prices[sample_assets].plot(ax=ax)
ax.set_title('Sample Asset Prices', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Price')
ax.legend(loc='upper left')
plt.tight_layout()
plt.show()

## 2. Feature Engineering

Calculate various price-based features used in momentum strategies.

In [ ]:
def calculate_returns(prices, periods=[1, 5, 21, 63, 126, 252]):
    """
    Calculate returns over multiple horizons.
    
    Parameters:
    -----------
    prices : pd.DataFrame
        Price data
    periods : list
        Return horizons in days
        
    Returns:
    --------
    dict
        Dictionary of return DataFrames
    """
    returns_dict = {}
    for period in periods:
        returns_dict[f'ret_{period}d'] = prices.pct_change(period)
    return returns_dict

# Calculate returns
returns_dict = calculate_returns(prices)
daily_returns = prices.pct_change()

print("Available return horizons:")
for key in returns_dict:
    print(f"  - {key}: shape {returns_dict[key].shape}")

In [ ]:
def calculate_volatility(returns, windows=[21, 63, 252]):
    """
    Calculate rolling volatility.
    
    Parameters:
    -----------
    returns : pd.DataFrame
        Daily returns
    windows : list
        Rolling window sizes
        
    Returns:
    --------
    dict
        Dictionary of volatility DataFrames
    """
    vol_dict = {}
    for window in windows:
        vol_dict[f'vol_{window}d'] = returns.rolling(window).std() * np.sqrt(252)
    return vol_dict

# Calculate volatility features
vol_dict = calculate_volatility(daily_returns)

print("Volatility features calculated.")

## 3. Signal Construction

Implement the 12-1 momentum signal: 12-month return excluding the most recent month.

In [ ]:
def calculate_momentum_signal(prices, lookback=252, skip=21):
    """
    Calculate 12-1 momentum signal.
    
    Parameters:
    -----------
    prices : pd.DataFrame
        Price data
    lookback : int
        Total lookback period (252 ≈ 12 months)
    skip : int
        Skip most recent period (21 ≈ 1 month)
        
    Returns:
    --------
    pd.DataFrame
        Momentum signal
    """
    # Calculate return from t-lookback to t-skip
    return prices.shift(skip) / prices.shift(lookback) - 1

# Calculate momentum signal
momentum = calculate_momentum_signal(prices)
print(f"Momentum signal shape: {momentum.shape}")
print(f"Non-null values from: {momentum.dropna(how='all').index[0]}")

In [ ]:
def rank_cross_sectional(signal):
    """
    Rank signal cross-sectionally (0 to 1 scale).
    
    Parameters:
    -----------
    signal : pd.DataFrame
        Raw signal values
        
    Returns:
    --------
    pd.DataFrame
        Cross-sectional ranks (0 = lowest, 1 = highest)
    """
    return signal.rank(axis=1, pct=True)

# Calculate cross-sectional ranks
momentum_rank = rank_cross_sectional(momentum)

# Visualize signal distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Raw momentum distribution
sample_date = momentum.dropna(how='all').index[-252]  # 1 year before end
axes[0].hist(momentum.loc[sample_date].dropna(), bins=30, edgecolor='black')
axes[0].set_title(f'Momentum Distribution ({sample_date.date()})', fontsize=12)
axes[0].set_xlabel('12-1 Momentum')
axes[0].set_ylabel('Frequency')

# Ranked momentum
axes[1].hist(momentum_rank.loc[sample_date].dropna(), bins=30, edgecolor='black')
axes[1].set_title(f'Ranked Momentum Distribution ({sample_date.date()})', fontsize=12)
axes[1].set_xlabel('Momentum Rank')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 4. Backtesting

Implement a simple backtesting framework to evaluate the momentum strategy.

In [ ]:
class MomentumBacktest:
    """
    Backtesting framework for momentum strategy.
    """
    
    def __init__(self, prices, signal, long_threshold=0.9, short_threshold=0.1, 
                 rebal_freq='M', transaction_cost=0.001):
        """
        Initialize backtest.
        
        Parameters:
        -----------
        prices : pd.DataFrame
            Price data
        signal : pd.DataFrame
            Trading signal (ranked)
        long_threshold : float
            Threshold for long positions
        short_threshold : float
            Threshold for short positions
        rebal_freq : str
            Rebalancing frequency
        transaction_cost : float
            One-way transaction cost
        """
        self.prices = prices
        self.signal = signal
        self.long_threshold = long_threshold
        self.short_threshold = short_threshold
        self.rebal_freq = rebal_freq
        self.transaction_cost = transaction_cost
        
        self.returns = prices.pct_change()
        self.portfolio_returns = None
        self.weights_history = None
        
    def generate_weights(self):
        """
        Generate portfolio weights based on signal.
        
        Returns:
        --------
        pd.DataFrame
            Portfolio weights
        """
        # Initialize weights
        weights = pd.DataFrame(0.0, index=self.signal.index, columns=self.signal.columns)
        
        # Long positions (top decile)
        long_mask = self.signal >= self.long_threshold
        n_long = long_mask.sum(axis=1)
        
        # Short positions (bottom decile)
        short_mask = self.signal <= self.short_threshold
        n_short = short_mask.sum(axis=1)
        
        # Equal weight within long/short buckets
        for date in weights.index:
            if n_long[date] > 0:
                weights.loc[date, long_mask.loc[date]] = 1.0 / n_long[date]
            if n_short[date] > 0:
                weights.loc[date, short_mask.loc[date]] = -1.0 / n_short[date]
        
        # Resample to rebalancing frequency (month-end)
        rebal_dates = weights.resample(self.rebal_freq).last().index
        weights_rebal = weights.loc[weights.index.isin(rebal_dates)]
        
        # Forward fill weights
        weights_final = weights_rebal.reindex(self.returns.index).ffill()
        
        self.weights_history = weights_final
        return weights_final
    
    def calculate_portfolio_returns(self, weights):
        """
        Calculate portfolio returns including transaction costs.
        
        Parameters:
        -----------
        weights : pd.DataFrame
            Portfolio weights
            
        Returns:
        --------
        pd.Series
            Portfolio returns
        """
        # Calculate gross returns
        gross_returns = (weights.shift(1) * self.returns).sum(axis=1)
        
        # Calculate turnover
        weight_changes = weights.diff().abs().sum(axis=1)
        turnover_costs = weight_changes * self.transaction_cost
        
        # Net returns
        net_returns = gross_returns - turnover_costs
        
        self.portfolio_returns = net_returns
        self.turnover = weight_changes
        
        return net_returns
    
    def run(self):
        """
        Run the backtest.
        
        Returns:
        --------
        pd.Series
            Portfolio returns
        """
        weights = self.generate_weights()
        returns = self.calculate_portfolio_returns(weights)
        return returns

In [ ]:
# Run backtest
backtest = MomentumBacktest(
    prices=prices,
    signal=momentum_rank,
    long_threshold=0.9,
    short_threshold=0.1,
    transaction_cost=0.001
)

portfolio_returns = backtest.run()

# Remove initial NaN period
portfolio_returns = portfolio_returns.dropna()

print(f"Backtest period: {portfolio_returns.index[0]} to {portfolio_returns.index[-1]}")
print(f"Number of trading days: {len(portfolio_returns)}")

## 5. Performance Analysis

Evaluate strategy performance using standard risk-adjusted metrics.

In [ ]:
def calculate_performance_metrics(returns, risk_free_rate=0.02):
    """
    Calculate comprehensive performance metrics.
    
    Parameters:
    -----------
    returns : pd.Series
        Strategy returns
    risk_free_rate : float
        Annual risk-free rate
        
    Returns:
    --------
    dict
        Performance metrics
    """
    # Annualization factor
    ann_factor = 252
    
    # Basic metrics
    total_return = (1 + returns).prod() - 1
    ann_return = (1 + total_return) ** (ann_factor / len(returns)) - 1
    ann_vol = returns.std() * np.sqrt(ann_factor)
    
    # Sharpe Ratio
    excess_return = ann_return - risk_free_rate
    sharpe = excess_return / ann_vol if ann_vol > 0 else 0
    
    # Drawdown analysis
    cum_returns = (1 + returns).cumprod()
    running_max = cum_returns.cummax()
    drawdown = (cum_returns - running_max) / running_max
    max_drawdown = drawdown.min()
    
    # Calmar Ratio
    calmar = ann_return / abs(max_drawdown) if max_drawdown != 0 else 0
    
    # Sortino Ratio
    downside_returns = returns[returns < 0]
    downside_vol = downside_returns.std() * np.sqrt(ann_factor)
    sortino = excess_return / downside_vol if downside_vol > 0 else 0
    
    # Win rate
    win_rate = (returns > 0).sum() / len(returns)
    
    # Profit factor
    gross_profits = returns[returns > 0].sum()
    gross_losses = abs(returns[returns < 0].sum())
    profit_factor = gross_profits / gross_losses if gross_losses > 0 else np.inf
    
    return {
        'Total Return': f'{total_return:.2%}',
        'Annual Return': f'{ann_return:.2%}',
        'Annual Volatility': f'{ann_vol:.2%}',
        'Sharpe Ratio': f'{sharpe:.2f}',
        'Sortino Ratio': f'{sortino:.2f}',
        'Max Drawdown': f'{max_drawdown:.2%}',
        'Calmar Ratio': f'{calmar:.2f}',
        'Win Rate': f'{win_rate:.2%}',
        'Profit Factor': f'{profit_factor:.2f}'
    }

# Calculate metrics
metrics = calculate_performance_metrics(portfolio_returns)

print("\n" + "="*50)
print("MOMENTUM STRATEGY PERFORMANCE METRICS")
print("="*50)
for key, value in metrics.items():
    print(f"{key:20} : {value:>12}")
print("="*50)

In [ ]:
# Plot equity curve and drawdown
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Cumulative returns
cum_returns = (1 + portfolio_returns).cumprod()
axes[0].plot(cum_returns.index, cum_returns.values, 'b-', linewidth=1.5)
axes[0].set_title('Momentum Strategy - Equity Curve', fontsize=14)
axes[0].set_ylabel('Cumulative Return')
axes[0].axhline(y=1, color='gray', linestyle='--', alpha=0.5)
axes[0].fill_between(cum_returns.index, 1, cum_returns.values, 
                     where=(cum_returns.values >= 1), alpha=0.3, color='green')
axes[0].fill_between(cum_returns.index, 1, cum_returns.values, 
                     where=(cum_returns.values < 1), alpha=0.3, color='red')

# Drawdown
running_max = cum_returns.cummax()
drawdown = (cum_returns - running_max) / running_max
axes[1].fill_between(drawdown.index, drawdown.values, 0, alpha=0.7, color='red')
axes[1].set_title('Drawdown', fontsize=14)
axes[1].set_ylabel('Drawdown')

# Rolling Sharpe
rolling_sharpe = portfolio_returns.rolling(252).mean() / portfolio_returns.rolling(252).std() * np.sqrt(252)
axes[2].plot(rolling_sharpe.index, rolling_sharpe.values, 'purple', linewidth=1.5)
axes[2].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[2].axhline(y=1, color='green', linestyle='--', alpha=0.5, label='Sharpe = 1')
axes[2].set_title('Rolling 1-Year Sharpe Ratio', fontsize=14)
axes[2].set_ylabel('Sharpe Ratio')
axes[2].set_xlabel('Date')
axes[2].legend()

plt.tight_layout()
plt.show()

## 6. Risk Analysis

Analyze strategy risk characteristics including factor exposures and tail risk.

In [ ]:
# Monthly returns analysis
monthly_returns = portfolio_returns.resample('M').apply(lambda x: (1+x).prod()-1)

# Create monthly returns heatmap
monthly_pivot = monthly_returns.to_frame('Returns')
monthly_pivot['Year'] = monthly_pivot.index.year
monthly_pivot['Month'] = monthly_pivot.index.month
monthly_heatmap = monthly_pivot.pivot(index='Year', columns='Month', values='Returns')

fig, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(monthly_heatmap, annot=True, fmt='.1%', cmap='RdYlGn', center=0, 
            linewidths=0.5, ax=ax, cbar_kws={'label': 'Return'})
ax.set_title('Monthly Returns Heatmap', fontsize=14)
ax.set_xlabel('Month')
ax.set_ylabel('Year')
plt.tight_layout()
plt.show()

In [ ]:
# Return distribution analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Daily returns histogram
axes[0].hist(portfolio_returns, bins=50, edgecolor='black', density=True, alpha=0.7)
axes[0].axvline(portfolio_returns.mean(), color='red', linestyle='--', label=f'Mean: {portfolio_returns.mean():.4f}')
axes[0].axvline(portfolio_returns.median(), color='green', linestyle='--', label=f'Median: {portfolio_returns.median():.4f}')
axes[0].set_title('Daily Returns Distribution', fontsize=12)
axes[0].set_xlabel('Daily Return')
axes[0].set_ylabel('Density')
axes[0].legend()

# QQ plot
from scipy import stats
stats.probplot(portfolio_returns, dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot (vs Normal Distribution)', fontsize=12)

plt.tight_layout()
plt.show()

# Calculate tail risk metrics
var_95 = portfolio_returns.quantile(0.05)
cvar_95 = portfolio_returns[portfolio_returns <= var_95].mean()
skewness = portfolio_returns.skew()
kurtosis = portfolio_returns.kurtosis()

print(f"\nTail Risk Metrics:")
print(f"  VaR (95%):  {var_95:.4f} ({var_95:.2%})")
print(f"  CVaR (95%): {cvar_95:.4f} ({cvar_95:.2%})")
print(f"  Skewness:   {skewness:.4f}")
print(f"  Kurtosis:   {kurtosis:.4f}")

In [ ]:
# Turnover analysis
turnover = backtest.turnover.resample('M').sum()

fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(turnover.index, turnover.values, width=20, alpha=0.7, edgecolor='black')
ax.axhline(turnover.mean(), color='red', linestyle='--', label=f'Mean: {turnover.mean():.2f}')
ax.set_title('Monthly Portfolio Turnover', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Turnover')
ax.legend()
plt.tight_layout()
plt.show()

print(f"\nTurnover Statistics:")
print(f"  Average Monthly Turnover: {turnover.mean():.2%}")
print(f"  Annual Turnover:          {turnover.mean() * 12:.2%}")

## Conclusions

### Key Findings

1. **Strategy Performance**: The momentum strategy demonstrates [positive/negative] risk-adjusted returns with a Sharpe ratio of approximately [X.XX].

2. **Risk Characteristics**: 
   - Maximum drawdown of [X%] occurred during [period]
   - The strategy shows [positive/negative] skewness indicating [left/right] tail risk

3. **Transaction Costs**: With 10bps one-way costs and monthly rebalancing, transaction costs reduce returns by approximately [X%] annually.

### Potential Improvements

- **Signal Enhancement**: Combine with other factors (value, quality) for diversification
- **Risk Management**: Implement volatility targeting or drawdown controls
- **Execution**: Consider optimized rebalancing schedules to reduce turnover